In [ ]:
import cv2
import scipy
import numpy as np
import matplotlib.pyplot as plt
import skimage.restoration
from skimage.restoration import estimate_sigma
import pywt
import bm3d
from collections import defaultdict
import os

def add_noise(original, sigma):
  np.random.seed(1)
  noise = np.random.normal(scale=sigma, size=original.shape) # scale = standard deviation
  return noise + original

def fast_estimate(noisy):
  N = noisy.shape[0]
  M = [[1, -2, 1],
  [-2, 4, -2],
  [1, -2, 1]]
  sigma = np.sum(np.abs(scipy.signal.convolve2d(noisy, M)))
  sigma = sigma * np.sqrt(np.pi/2)/(6*(N-2)**2)
  return sigma

def test_dataset(folder_path):
  """
  Tests non-linear means denoising on all the images in folder_path, and returns the total MSE for each experiment.

  Runs each experiment with sigma = 10, 25, 40
  """
  mses = 0
  for file in os.listdir(folder_path):
    # Load image + add noise
    original = cv2.imread(folder_path + '/' + file)
    original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    for sigma in (10, 25, 40):
      noisy = add_noise(original, sigma)
      noisy = noisy / noisy.max() #normalizes data in range 0 - 255
      noisy = 255 * noisy
      noisy = noisy.astype(np.uint8)
      denoised = cv2.fastNlMeansDenoising(noisy, None, ((sigma/2)*np.sqrt(2*np.log(original.size)))/2, 7, 21)
      mses += (((original-denoised)**2).mean(axis = None))
  print(mses)
  return mses    
    

test_dataset('test_images')